In [14]:
# Install the package in development mode if needed
# !pip install -e '.[mcp]'

import asyncio
import os
import sys
import logging
from pathlib import Path
import json

# Import the MCP components
from napistu.mcp.server import create_server, start_server
from napistu.mcp.components import documentation, codebase, tutorials, execution

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("napistu")

# Helper function to run async code in Jupyter
async def run_async(coro):
    return await coro

# Create a dummy session context for execution components
session_context = {}
object_registry = {}

In [ ]:
from napistu.mcp.profiles import get_profile
# define the types of assets to load
profile = get_profile("full")

In [18]:
# register the relevant components
mcp_server = create_server(profile)
# initialize the relevant components
live_server = start_server(mcp_server)

In [23]:
live_server.cr_running

False